In [1]:
from konlpy.tag import Okt # 트위터 한국어 처리기 - 일상 생활에 등장하는 단어에 유용
from konlpy.tag import Komoran

In [3]:
import json

def open_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

In [4]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time

# 측정 시작
start = time.time()

# 리뷰 파일 읽어오기
print('1) 말뭉치 데이터 읽기 시작')
train = open_json("train_game.json")
print(len(train)) # 리뷰 데이터 전체 개수
print('1) 말뭉치 데이터 읽기 완료: ', time.time() - start)

1) 말뭉치 데이터 읽기 시작
863
1) 말뭉치 데이터 읽기 완료:  0.02299952507019043


In [5]:
train[0]

{'id': 2518043,
 'sentence': '퀘스트 사냥꾼은 대체로 컨트롤 성향 덱에 더 강한 모습을 보이나, 컨트롤 성기사엔 조금 약하며 나가 악마사냥꾼에도 다소 밀리는 편이다.',
 'tokens': [{'start': 0,
   'length': 7,
   'sub': '퀘스트 사냥꾼',
   'facet': '직업',
   'term_id': 336714,
   'sense_no': 1}],
 'sense_no': 1,
 'source': {'uri': 'https://www.inven.co.kr/webzine/news/?news=271351&site=hs',
  'text': '지난 4월 27일 23.0.3 패치로 높은 승률을 보이던 카드가 일부 조정되었다. 가장 큰 조정을 받은 직업은 드루이드로, [기적같은 성장]의 비용이 1 늘고 [카자쿠산]의 조건이 이 게임에서 다른 용족을 넷 냈다면 보물 더미 덱을 만드는 것으로 바뀌었다.\n또한 전 퀘스트 [항구 습격]의 세 번째 단계까 해적을 셋 내야 하는 것으로 바뀌어 속도가 늦춰지고, [복어주먹]은 생명력이 1 줄었다. \n램프 드루이드의 버티기 및 덱 구성이 어려워지고 퀘스트 전사의 속도가 줄어들며 여러 덱이 상위로 올라왔다. 특히 나가 악마사냥꾼이 가장 좋은 모습을 보여주고 있으며 이에 맞춰 컨트롤 전사와 컨트롤 성기사, 퀘스트 사냥꾼도 좋은 덱으로 꼽히고 있다. \n나가 악마사냥꾼\n빠르게 몰아치는 강력한 어그로 유형\n[흉포한 갈퀴창날][보물 경비병] 등 나가를 쓰는 어그로 유형의 악마사냥꾼이 메타의 새로운 중심이 되었다. 지난 23.0.3 패치에서 [복어주먹]의 생명력이 1 줄어든 것 외에는 너프가 없었고, 여기에 강력했던 드루이드의 버티기 능력이 약화되어 악마사냥꾼의 승률이 올랐다.\n1마나 1/3이며 명예로운 일격으로 적 영웅에게 내 영웅의 공격력만큼 피해를 주는 무기 [공포감옥 전투검]이 초반부터 상대의 명치를 위협하는데, [분노][연속 타격][혼돈의 일격]과 같은 영웅 공격력 획

In [7]:
# 문장단위로 명사만 추출해 학습 입력 데이터로 만듬
print('2) 형태소에서 명사만 추출 시작')
komoran = Komoran()
docs = [komoran.nouns(sentence["sentence"]) for sentence in train]
print('2) 형태소에서 명사만 추출 완료: ', time.time() - start)

2) 형태소에서 명사만 추출 시작
2) 형태소에서 명사만 추출 완료:  11.982598304748535


In [13]:
print(docs[0])

['퀘스트', '사냥', '컨트롤', '성향', '모습', '컨트롤', '성기사', '악마', '사냥', '편']


In [8]:
# word2vec 모델 학습
print('3) word2vec 모델 학습 시작')
model = Word2Vec(sentences=docs, vector_size=200, window=4, min_count=2, sg=1) # skip-gram
print('3) word2vec 모델 학습 완료: ', time.time() - start)

3) word2vec 모델 학습 시작
3) word2vec 모델 학습 완료:  14.454477071762085


In [ ]:
# 모델 저장
print('4) 학습된 모델 저장 시작')
# model.save('nvmc.model')
print('4) 학습된 모델 저장 완료: ', time.time() - start)

# 학습된 말뭉치 개수, 코퍼스 내 전체 단어 개수
print("corpus_count : ", model.corpus_count)
print("corpus_total_words : ", model.corpus_total_words)

4) 학습된 모델 저장 시작
4) 학습된 모델 저장 완료:  20.752155780792236
corpus_count :  863
corpus_total_words :  7576


In [ ]:
# 모델 로딩
# model = Word2Vec.load('nvmc.model')
print("corpus_total_words : ", model.corpus_total_words)

corpus_total_words :  7576


In [16]:
# 가장 유사한 단어 추출
print(model.wv.most_similar("카드", topn=5))
print(model.wv.most_similar("사냥", topn=5))

[('때', 0.74336177110672), ('거', 0.724134087562561), ('것', 0.7167823910713196), ('스키', 0.7071779370307922), ('수', 0.69302898645401)]
[('드', 0.6120314002037048), ('거', 0.610780656337738), ('때', 0.6032494902610779), ('수', 0.6031776666641235), ('쿠', 0.6013575196266174)]
